In [1]:
# dataExchange_attributeConstruction
# -*- coding:utf-8 -*-
# 目标：属性构造和洗浴事件筛选
# 需要构造的属性如下：
#热水事件	起始数据编号	终止数据编号	开始时间（begin_time）
# 根据日志判断是否为洗浴（1表示是，0表示否）	洗浴时间点	总用水时长（w_time）	
# 总停顿时长（w_pause_time）	平均停顿时长（avg_pause_time）	停顿次数（pause）	
# 用水时长（use_water_time）	用水/总时长（use_water_rate）	总用水量（w_water）
# 平均水流量（water_rate）	水流量波动（flow_volatility）	停顿时长波动（pause_volatility）

import pandas as pd
import numpy as np
from pandas import DataFrame,Series


inputfile = 'dataExchange_divideEvent.xlsx'
data = pd.read_excel(inputfile)
# len(data)# 7696
inputfile1 = 'data_guiyue.xlsx'
data1 = pd.read_excel(inputfile1)
x = pd.merge(data1,data[[u'事件编号']],left_index = True, right_index=True,how='outer')
# 连接'data_guiyue.xlsx'和 'dataexchange_divideEvent.xlsx'的后两列,因为，属性规约里面包含水流量为0的数值，
# 后一个表中，只含有水流量不为0的值，需要将两边进行连接，获知规约后的数据中的数据所属的事件数,利用处理后的数开始进行数据构造工作
x.head()#18840
x.to_excel('data_for_attr_const.xlsx')


In [2]:
df = pd.read_excel('data_for_attr_const.xlsx')
# 将数据划分成一次用水事件！（*****）
#-----第*1*步-----做基本处理，获取用于构造属性的数据表
# 将数据划分成一次用水事件！（*****）
# 思路：获取每个事件的序号值所在的最小的index值和最大的index值，然后将其连接，
# 即： 去掉不同事件间的“事件编号”为空的值，保留同一个事件内的“事件编号”为空的值
l=list(df[u'事件编号'])
Adf = DataFrame([], columns = df.columns)# 创建一个空列表
pos=-1
MX = int(df[u'事件编号'].max())
for j in range(MX):
    y = []
    for i in range(l.count(j+1)):
        pos=l.index(j+1,pos+1)
        y.append(pos)
    a = min(y)
    b = max(y)
    temp = df.iloc[a:b+1,:]
    Adf = pd.concat([Adf,temp])
Adf[u'事件编号'].fillna(method='ffill',inplace = True) # 向后填充，填上了事件编号的空值
Adf[[u'水流量']] = Adf[[u'水流量']].astype('float64')
Adf.to_excel('1TimeWaterDivide.xlsx')
Adf.head()


发生时间 开关机状态 加热中 保温中  实际温度 热水量   水流量 加热剩余时间 当前设置温度  事件编号
2   2014-10-19 07:01:56     关   关   关  30°C  0%   8.0    0分钟   50°C   1.0
56  2014-10-19 07:38:16     关   关   关  30°C  0%   8.0    0分钟   50°C   2.0
381 2014-10-19 09:46:38     关   关   关  29°C  0%  16.0    0分钟   50°C   3.0
382 2014-10-19 09:46:40     关   关   关  29°C  0%  13.0    0分钟   50°C   3.0
383 2014-10-19 09:46:58     关   关   关  29°C  0%   0.0    0分钟   50°C   3.0

In [3]:
#-----第*2*步-----建立存放构造的属性的表
df2 = pd.read_excel('1TimeWaterDivide.xlsx')
df2['gap'] = df2[u'发生时间'].diff()
MX = int(df2[u'事件编号'].max())# 获取当前事件数172
fdf2 = DataFrame([], index = range(1,MX+1))# 创建一个空列表 用来存放属性规约结果


In [4]:
#-----第*3*步-----开始构造属性
#构造属性第一部分：包含用水事件开始编号、用水事件结束编号、用水开始时间、用水结束时间、用水总时间、用水间隔时间、一次用水期间“水流量”为0的记录数
fdf2.index.names = ['eventNUM']

fdf2['stDataIndex'] = np.nan #用水事件开始编号
fdf2['edDataIndex'] = np.nan #用水事件结束编号
fdf2['stUseTime'] = np.nan #用水开始时间
fdf2['enUseTime'] = np.nan #用水结束时间
fdf2['AllUseTime'] = np.nan #用水总时间
fdf2['gapTimes'] = np.nan #用水间隔时间
fdf2['stopLines'] = np.nan # 一次用水期间“水流量”为0的记录数

ds = pd.Timedelta(seconds = 2) # 发送阈值时间设置为2秒
pos=-1# 标记变量     ******
l=list(df2[u'事件编号'])

# 以下空列表均是暂时存储数据
stUI = []
enUI = []
startU = []
endU = []
allUT = []
gapTS = []
stopLines = []
for j in range(MX):
    # 计算事件编号j在列表中的最开始出现的index和最后出现的index
    y = []# 存储时间编号j出现的所有的index（临时存储）
    for i in range(l.count(j+1)):
        pos=l.index(j+1,pos+1)
        y.append(pos)
    a = min(y)#出现值i最小的index
    b = max(y)#出现值i最大的index
    
    #记录一次用水的开始事件编号、结束事件编号
    stui= df2.index[a]
    enui= df2.index[b]
    stUI.append(stui)
    enUI.append(enui)
    
    #记录一次用水开始时间、结束时间
    stu = df2.iloc[a,0]-ds/2 # 设置用水开始时间=起始数据时间-发送阈值/2
    enu = df2.iloc[b,0]+ds/2 # 设置结束用水时间=结束数据时间+发送阈值/2
    startU.append(stu)
    endU.append(enu)
    
    #记录一次用水总时长
    allut = enu-stu
    allUT.append(allut)
    
    #判断停顿的行数（一次事件中水流量为0的记录的条数）
    stpts = df2.iloc[a:b+1,:]
    f =list(stpts[u'水流量'])
    stpt = f.count(0)#计算空值的个数
    stopLines.append(stpt)
  
    n =0   
    #判断停顿次数，中间有一个或多个空值均算作一次停顿
    if a==b :
        n=0
    else:
        tgap = df2.iloc[a:b+1,:]
        for t in range(a,b):
            g = tgap.ix[df2.index[t],[u'水流量']].values
            g1 = tgap.ix[df2.index[t+1],[u'水流量']].values
            if g1 == 0 and g!= 0:
                n+=1
    gapTS.append(n)
    
fdf2['stUseTime'] = startU
fdf2['enUseTime'] = endU
fdf2['stDataIndex'] = stUI
fdf2['edDataIndex'] = enUI
fdf2['AllUseTime'] = allUT
fdf2['gapTimes'] =gapTS
# 将一次用水事件的总时间转成以秒计
fdf2['AllUseTime(s)']= fdf2['AllUseTime']/np.timedelta64(1, 's')
fdf2['stopLines']= stopLines

D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:64: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [5]:
# 构造属性第二部分：获取用水的时间点的小时数，即洗浴时间点
fdf2['WashHour'] = np.nan

for i in range(1,len(fdf2['stUseTime'])+1):
    temp = fdf2.ix[i,'stUseTime'].strftime('%Y-%m-%d %H:%M:%S')# 将时间格式转成字符串,通过字符串截取获得时间点
    c = temp[11:13]
    d =int(c)
    fdf2.ix[i,'WashHour'] = d

D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """


In [6]:
# 构造属性第三部分：确定一次用水时间和停顿时间、总用水量、水流量波动、平均用户水量
fdf2['UseTime'] = np.nan # 用水时间
fdf2['GapTime'] = np.nan # 停顿时间
fdf2['w_water'] = np.nan # 总用水量
fdf2['flow_volatility'] = np.nan # 水流量波动
fdf2['water_rate'] = np.nan #平均用水量


# 计算停顿时间（具体所有方法见time_gap_compute.py，此处采用的是第二种方法)
def gap_time_2(y):
    gap_time_2 = pd.Timedelta(seconds = 0)
    templist = []
    allgap = 0
    # 获取一次用水时间中间隔的行编号
    for i in range(len(y)):
        if y.iloc[i,6] == 0:
            templist.append(i)
    # 计算停顿时间    
    if (len(y) ==1) | (templist==[]):# 如果该事件只有一个或者两个非零用水记录，直接让用水停顿时间等于0
        gap_time_2 = gap_time_2
    else:
        for j in templist:# 采用书上公式：每条用水数据时长的和=（和上条数据的间隔时间/2+和下条数据的间隔时间/2）的和
            gap_time_2 = gap_time_2 + y.iloc[j,-1]/2 + y.iloc[j+1,-1]/2
    gap_time = gap_time_2/np.timedelta64(1,'s')
    

    return gap_time

# 计算用水时间 和 总用水量
def use_time(y):
    send_time = pd.Timedelta(seconds = 2) # 定义信息发送延迟时间为2秒
    use_time = pd.Timedelta(seconds = 0)
    templist = []
    w_water = 0 # 记录总用水量
    # 获取一次用水时间用水的行编号
    LASTIME = []#记录每个用水记录的持续用水时间
    for i in range(len(y)):# 将所有用水量不为0的记录的行号进行记录
        if y.iloc[i,6] != 0:
            templist.append(i)
    if len(y) ==1:# 如果用水量不为0的仅为1条，则用水时间为发送时间，用水持续时间为发送时间
        use_time = send_time
        lastime = send_time
        w_water = lastime/np.timedelta64(1,'s')* y.iloc[0,6]# 该次用水量为用水持续时间*水流量
        LASTIME.append(lastime/np.timedelta64(1,'s'))
    else:
        lastime = pd.Timedelta(seconds = 0)#用水时间=每条用水数据时长的和=（和上条数据的时间间隔/2+和下条数据的时间间隔/2）的和
        for j in templist:
            if j == 0:# 每一个用水事件开始时刻的用水持续时间=和下条数据的时间间隔/2+发送时间/2
                lastime = y.iloc[j+1,-1]/2 + send_time/2
            elif j == (len(y)-1):  # 每一个用水事件最后时刻的用水持续时间=和上条数据的时间间隔/2+发送时间/2
                lastime = y.iloc[j,-1]/2 + send_time/2
            else:
                lastime = y.iloc[j,-1]/2 + y.iloc[j+1,-1]/2
            use_time = use_time + lastime
            w_water = w_water + lastime/np.timedelta64(1,'s')* y.iloc[j,6]
            LASTIME.append(lastime/np.timedelta64(1,'s'))
      
    usetime = use_time/np.timedelta64(1,'s')
    avg = w_water/usetime# 平均水流量=总用水量/总用水时间
    
    allwater = 0 # 计算水流量波动 = sum((单次水流的值-平均水流量)**2*持续时间)/总用水时间
    if len(y) ==1:
        allwater = (y.iloc[0,6]-avg)**2
    else:
        for i in range(len(templist)):
            allwater = allwater + LASTIME[i]*(y.iloc[templist[i],6]-avg)**2 
    flow_vola = allwater/100/usetime# 水流量波动 # 此处除以100为了让数字看起来正常点，符合原作者给出的配书中给的数据值
        
    return usetime, w_water/100, avg, flow_vola


useTIME = []# 用水时间
gapTIME = []# 间隔时间
w_wat = []# 总用水量
flow_volatility = []# 水流量波动
avg_water_rate = [] # 平均水流量
for n in range(1,int(MX+1)):
    gp= gap_time_2(df2[df2[u'事件编号'] == n])
    use,w_water,avg,flow_vola = use_time(df2[df2[u'事件编号'] == n])
    gapTIME.append(gp)
    useTIME.append(use)
    w_wat.append(w_water)
    flow_volatility.append(flow_vola)
    avg_water_rate.append(avg)
    
fdf2['GapTime'] = gapTIME  
fdf2['UseTime'] = useTIME # fdf2['AllUseTime(s)']- fdf2['GapTime']
fdf2['w_water'] = w_wat
fdf2['water_rate'] = avg_water_rate
fdf2['flow_volatility'] = flow_volatility 

fdf2.head()

stDataIndex  edDataIndex           stUseTime           enUseTime  \
eventNUM                                                                     
1                   2            2 2014-10-19 07:01:55 2014-10-19 07:01:57   
2                  56           56 2014-10-19 07:38:15 2014-10-19 07:38:17   
3                 381          384 2014-10-19 09:46:37 2014-10-19 09:47:16   
4                 404          404 2014-10-19 11:50:16 2014-10-19 11:50:18   
5                 407          407 2014-10-19 13:56:20 2014-10-19 13:56:22   

         AllUseTime  gapTimes  stopLines  AllUseTime(s)  WashHour  UseTime  \
eventNUM                                                                     
1          00:00:02         0          0            2.0       7.0      2.0   
2          00:00:02         0          0            2.0       7.0      2.0   
3          00:00:39         1          1           39.0       9.0     21.5   
4          00:00:02         0          0            2.0      11.0      2.0   
5          00:00:02         0          0            2.0      13.0      2.0   

          GapTime  w_water  flow_volatility  water_rate  
eventNUM                                                 
1             0.0     0.16         0.000000    8.000000  
2             0.0     0.16         0.000000    8.000000  
3            17.5     3.52         0.111174   16.372093  
4             0.0     0.44         0.000000   22.000000  
5             0.0     0.16         0.000000    8.000000

In [7]:
# 构造属性第四部分：计算停顿时长波动
fdf2['pause_volatility'] = np.nan
pause_volatility = [] # 停顿时长波动
# 获取事件编号为eventnum的停顿次数
def get_gaptimes(eventnum):
    return fdf2['gapTimes'][eventnum] 

# 计算得出停顿时间和停顿时长波动（具体所有方法见time_gap_compute.py，此处采用的是第一种方法），因为这样可以算出单次停顿的具体时间
# 停顿时长波动=sum((单次停顿时长-平均停顿时长)**2*持续时间)/总停顿时间
def gap_time(x,gapTimes):
    gap_time = pd.Timedelta(seconds = 0)
    GAPTIME = []
    pause_vola = 0
    if len(x) ==1:
        gap_time = gap_time
        pause_vola = 0
    else:
        i=0
        tempdf = DataFrame([])
        while i< len(x)-1:
            i= i+1
            if (x.loc[x.index[i],u'水流量']) == 0:#若第i条水量为0
                tempdf = tempdf.append(x.ix[x.index[i],:])# 存储所有水流量为0的数据的记录
                if (x.loc[x.index[i+1],u'水流量']) != 0:# 若第i+1条水量不为0，说明这是一次停顿的结束
                    start = tempdf.loc[tempdf.index[0],u'发生时间']    # 计算该次停顿的开始时刻
                    end = tempdf.loc[tempdf.index[-1],u'发生时间']     # 计算该次停顿的结束时刻
                    b = list(x.index).index(tempdf.index[0])-1# 获取该次停顿发生的前一个用水量非0的记录的index
                    start_gap = x.iloc[b,0]
                    c = list(x.index).index(tempdf.index[-1])+1# 获取该次停顿结束的后一个用水量非0的记录的index
                    end_gap = x.iloc[c,0]
#                     print start_gap,start,end,end_gap
                    tempdf = DataFrame([]) # 清空tempdf，以记录下一次停顿时间
                    t1 = (start-start_gap)/2 # 停顿开始时与上一条非零数据的时间间隔/2
                    t2 = (end_gap-end)/2# 停顿结束时与下一条非零数据的时间间隔/2
                    t3 = end-start # 中间停顿时间
                    t = t1+t2+t3
#                     print t1,t2,t3,t
                    gap_time = gap_time+t
                    GAPTIME.append(t/np.timedelta64(1,'s'))#将每一小段的停顿时间存入，以计算停顿时长波动
    gap_time = gap_time/np.timedelta64(1,'s') # 将停顿时间转成“秒”
    # 计算平均停顿时间 = 总停顿时间/停顿次数
    if gapTimes != 0:# 若停顿次数不为0
        avg = gap_time/gapTimes 
    else:# 若停顿次数为0，则平均停顿时间等于停顿的时间
        avg = gap_time
    
    Allgap=0# 所有的停顿时间
    for i in range(len(GAPTIME)):
        Allgap = Allgap + GAPTIME[i]*(GAPTIME[i]-avg)**2
    pause_vola = Allgap/gap_time# 水流量波动
    return gap_time, pause_vola

for n in range(1,MX+1):
    gp, pause_vola = gap_time(df2[df2[u'事件编号'] == n], get_gaptimes(n))   
    pause_volatility.append(pause_vola)

fdf2['pause_volatility'] = pause_volatility 
fdf2.head()

D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in double_scalars
D:\Anaconda2\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


stDataIndex  edDataIndex           stUseTime           enUseTime  \
eventNUM                                                                     
1                   2            2 2014-10-19 07:01:55 2014-10-19 07:01:57   
2                  56           56 2014-10-19 07:38:15 2014-10-19 07:38:17   
3                 381          384 2014-10-19 09:46:37 2014-10-19 09:47:16   
4                 404          404 2014-10-19 11:50:16 2014-10-19 11:50:18   
5                 407          407 2014-10-19 13:56:20 2014-10-19 13:56:22   

         AllUseTime  gapTimes  stopLines  AllUseTime(s)  WashHour  UseTime  \
eventNUM                                                                     
1          00:00:02         0          0            2.0       7.0      2.0   
2          00:00:02         0          0            2.0       7.0      2.0   
3          00:00:39         1          1           39.0       9.0     21.5   
4          00:00:02         0          0            2.0      11.0      2.0   
5          00:00:02         0          0            2.0      13.0      2.0   

          GapTime  w_water  flow_volatility  water_rate  pause_volatility  
eventNUM                                                                   
1             0.0     0.16         0.000000    8.000000               NaN  
2             0.0     0.16         0.000000    8.000000               NaN  
3            17.5     3.52         0.111174   16.372093               0.0  
4             0.0     0.44         0.000000   22.000000               NaN  
5             0.0     0.16         0.000000    8.000000               NaN

In [8]:
# 构造属性第五部分：一次用水事件中，用水时长的所占比重
fdf2['use_water_rate'] = fdf2['UseTime']/fdf2['AllUseTime(s)']
# 平均停顿时间
# fdf2['avg_pause_time'] = fdf2['GapTime']/fdf2['gapTimes']# 不能这样除，除数可能为0
fdf2['avg_pause_time'] = np.nan
for i in range(1,len(fdf2)+1):
    if fdf2.ix[i,['gapTimes']].values[0] != 0:
        fdf2.ix[i,['avg_pause_time']] = fdf2.ix[i,['GapTime']].values[0] / fdf2.ix[i,['gapTimes']].values[0]
    else :
        fdf2.ix[i,['avg_pause_time']] = 0

fdf2.head()

stDataIndex  edDataIndex           stUseTime           enUseTime  \
eventNUM                                                                     
1                   2            2 2014-10-19 07:01:55 2014-10-19 07:01:57   
2                  56           56 2014-10-19 07:38:15 2014-10-19 07:38:17   
3                 381          384 2014-10-19 09:46:37 2014-10-19 09:47:16   
4                 404          404 2014-10-19 11:50:16 2014-10-19 11:50:18   
5                 407          407 2014-10-19 13:56:20 2014-10-19 13:56:22   

         AllUseTime  gapTimes  stopLines  AllUseTime(s)  WashHour  UseTime  \
eventNUM                                                                     
1          00:00:02         0          0            2.0       7.0      2.0   
2          00:00:02         0          0            2.0       7.0      2.0   
3          00:00:39         1          1           39.0       9.0     21.5   
4          00:00:02         0          0            2.0      11.0      2.0   
5          00:00:02         0          0            2.0      13.0      2.0   

          GapTime  w_water  flow_volatility  water_rate  pause_volatility  \
eventNUM                                                                    
1             0.0     0.16         0.000000    8.000000               NaN   
2             0.0     0.16         0.000000    8.000000               NaN   
3            17.5     3.52         0.111174   16.372093               0.0   
4             0.0     0.44         0.000000   22.000000               NaN   
5             0.0     0.16         0.000000    8.000000               NaN   

          use_water_rate  avg_pause_time  
eventNUM                                  
1               1.000000             0.0  
2               1.000000             0.0  
3               0.551282            17.5  
4               1.000000             0.0  
5               1.000000             0.0

In [9]:
# 属性构造结束，将各列重命名，然后保存
fdf2.rename(columns={'stDataIndex':u'起始数据编号','edDataIndex':u'终止数据编号','stUseTime':u'开始时间','enUseTime':u'结束时间',\
                  'AllUseTime':u'总用水时长（w_time）', 'gapTimes':u'停顿次数（pause）','stopLines':u'停顿行数','AllUseTime(s)':\
                     u'所有使用时间总用水时长（s）','WashHour':u'洗浴时间点','UseTime':u'用水时长（use_water_time）','GapTime':\
                    u'总停顿时长（w_pause_time）','w_water':u'总用水量（w_water）','flow_volatility':u'水流量波动（flow_volatility）',\
                   'water_rate':u'平均水流量（water_rate）','use_water_rate':u'用水/总时长（use_water_rate）','avg_pause_time':\
                    u'平均停顿时长（avg_pause_time）','pause_volatility':u'停顿时长波动（pause_volatility）'},inplace='True' )

fdf2.index.name=u'事件编号'
fdf2.to_excel('attrConst_results.xlsx')
fdf2

起始数据编号  终止数据编号                开始时间                结束时间 总用水时长（w_time）  \
事件编号                                                                         
1          2       2 2014-10-19 07:01:55 2014-10-19 07:01:57      00:00:02   
2         56      56 2014-10-19 07:38:15 2014-10-19 07:38:17      00:00:02   
3        381     384 2014-10-19 09:46:37 2014-10-19 09:47:16      00:00:39   
4        404     404 2014-10-19 11:50:16 2014-10-19 11:50:18      00:00:02   
5        407     407 2014-10-19 13:56:20 2014-10-19 13:56:22      00:00:02   
6        410     593 2014-10-19 15:34:38 2014-10-19 15:48:47      00:14:09   
7        612     618 2014-10-19 15:55:14 2014-10-19 15:55:30      00:00:16   
8        659     765 2014-10-19 17:23:20 2014-10-19 17:34:37      00:11:17   
9        772     922 2014-10-19 17:46:20 2014-10-19 17:55:05      00:08:45   
10      1025    1134 2014-10-19 21:32:44 2014-10-19 21:42:02      00:09:18   
11      1139    1139 2014-10-19 21:47:04 2014-10-19 21:47:06      00:00:02   
12      1141    1143 2014-10-19 21:54:29 2014-10-19 21:54:39      00:00:10   
13      1258    1258 2014-10-20 06:59:32 2014-10-20 06:59:34      00:00:02   
14      1345    1424 2014-10-20 07:22:53 2014-10-20 07:38:49      00:15:56   
15      1444    1445 2014-10-20 13:33:53 2014-10-20 13:33:57      00:00:04   
16      1450    1460 2014-10-20 19:03:40 2014-10-20 19:04:10      00:00:30   
17      1481    1532 2014-10-20 19:13:44 2014-10-20 19:17:52      00:04:08   
18      1571    1632 2014-10-20 19:47:58 2014-10-20 19:53:36      00:05:38   
19      1641    1641 2014-10-20 20:18:10 2014-10-20 20:18:12      00:00:02   
20      1659    1760 2014-10-20 21:46:21 2014-10-20 21:55:12      00:08:51   
21      1876    1876 2014-10-20 23:02:12 2014-10-20 23:02:14      00:00:02   
22      1879    1879 2014-10-21 06:43:36 2014-10-21 06:43:38      00:00:02   
23      1881    1883 2014-10-21 07:49:06 2014-10-21 07:49:14      00:00:08   
24      2068    2207 2014-10-21 16:04:17 2014-10-21 16:14:14      00:09:57   
25      2220    2381 2014-10-21 16:23:06 2014-10-21 16:41:12      00:18:06   
26      2385    2385 2014-10-21 17:37:53 2014-10-21 17:37:55      00:00:02   
27      2451    2451 2014-10-21 18:37:56 2014-10-21 18:37:58      00:00:02   
28      2454    2528 2014-10-21 18:49:41 2014-10-21 18:56:24      00:06:43   
29      2533    2533 2014-10-21 19:01:42 2014-10-21 19:01:44      00:00:02   
30      2546    2615 2014-10-21 19:50:01 2014-10-21 19:55:12      00:05:11   
...      ...     ...                 ...                 ...           ...   
143    16061   16162 2014-11-07 20:44:22 2014-11-07 20:54:54      00:10:32   
144    16165   16171 2014-11-07 21:00:54 2014-11-07 21:01:22      00:00:28   
145    16186   16235 2014-11-07 21:08:02 2014-11-07 21:18:08      00:10:06   
146    16237   16364 2014-11-07 21:47:02 2014-11-07 21:59:01      00:11:59   
147    16532   16553 2014-11-08 08:01:05 2014-11-08 08:02:42      00:01:37   
148    16563   16563 2014-11-08 08:23:05 2014-11-08 08:23:07      00:00:02   
149    16565   16581 2014-11-08 08:46:21 2014-11-08 08:49:17      00:02:56   
150    16632   16632 2014-11-08 11:47:35 2014-11-08 11:47:37      00:00:02   
151    16634   16634 2014-11-08 12:15:36 2014-11-08 12:15:38      00:00:02   
152    16745   16745 2014-11-08 18:53:28 2014-11-08 18:53:30      00:00:02   
153    16747   16752 2014-11-08 20:08:24 2014-11-08 20:08:40      00:00:16   
154    16788   16900 2014-11-08 22:05:49 2014-11-08 22:14:03      00:08:14   
155    16951   17047 2014-11-08 22:36:26 2014-11-08 22:50:05      00:13:39   
156    17212   17226 2014-11-09 07:38:03 2014-11-09 07:40:40      00:02:37   
157    17276   17276 2014-11-09 07:51:59 2014-11-09 07:52:01      00:00:02   
158    17507   17539 2014-11-09 21:06:23 2014-11-09 21:08:23      00:02:00   
159    17764   17764 2014-11-10 07:04:35 2014-11-10 07:04:37      00:00:02   
160    17766   17767 2014-11-10 09:09:18 2014-11-10 09:09:22      00:00:04   
161    17795   17796 2014-11

In [10]:
#-----第*4*步-----接下来进行数据处理和筛选得到"候选洗浴事件"，用于接下来的模型构建
# 去掉  用水时长小于100秒
# 去掉 总用水时长小于120秒
# 去掉 一次用水事件中总用水量(纯热水)小于10升
data_filter = fdf2[(fdf2[u'所有使用时间总用水时长（s）']>=120)  & (fdf2[u'总用水量（w_water）']>=10) & (fdf2[u'用水时长（use_water_time）']>=100)]
data_filter.to_excel('data_filter.xlsx')
# data_filter.iloc[:3]
data_filter.head(3)

起始数据编号  终止数据编号                开始时间                结束时间 总用水时长（w_time）  \
事件编号                                                                         
6        410     593 2014-10-19 15:34:38 2014-10-19 15:48:47      00:14:09   
8        659     765 2014-10-19 17:23:20 2014-10-19 17:34:37      00:11:17   
9        772     922 2014-10-19 17:46:20 2014-10-19 17:55:05      00:08:45   

      停顿次数（pause）  停顿行数  所有使用时间总用水时长（s）  洗浴时间点  用水时长（use_water_time）  \
事件编号                                                                   
6               1     2           849.0   15.0                 835.0   
8               2     3           677.0   17.0                 506.0   
9               0     0           525.0   17.0                 525.0   

      总停顿时长（w_pause_time）  总用水量（w_water）  水流量波动（flow_volatility）  \
事件编号                                                               
6                    14.0         337.90                0.637375   
8                   171.0         150.62                1.788448   
9                     0.0         134.16                0.841975   

      平均水流量（water_rate）  停顿时长波动（pause_volatility）  用水/总时长（use_water_rate）  \
事件编号                                                                        
6             40.467066                       0.0                0.983510   
8             29.766798                    2209.0                0.747415   
9             25.554286                       NaN                1.000000   

      平均停顿时长（avg_pause_time）  
事件编号                          
6                       14.0  
8                       85.5  
9                        0.0